# Installing MASE (again)

Run the block below to install MASE in the current Colab runtime

In [1]:
git_token = "YOUR_GIT_TOKEN"
short_code = "YOUR_SHORT_CODE"

# Check the current python version (It should be using Python 3.10) and update pip to the latest version.
!python --version
!python -m pip install --user --upgrade pip

# TODO: temporarily clone from prep branch - delete before release
# Clone MASE with your short code branch (notice your short code branch must be branch off from the coursweok-prep branch before)
!git clone -b {short_code} https://{git_token}@github.com/JianyiCheng/mase-tools.git

# Install requirements
!python -m pip install -r ./mase-tools/machop/requirements.txt

# Change working directory to machop
%cd ./mase-tools/machop/

Python 3.10.12
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.1 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Cloning into 'mase-tools'...
remote: Enumerating objects: 14069, done.
remote: Counting objects: 100% (1089/1089), done.
remote: Compressing objects: 100% (545/545), done.
remote: Total 14069 (delta 557), reused 930 (delta 458), pack-reused 12980
Receiving objects: 100% (14069/14069), 14.37 MiB | 13.11 MiB/s, done.
Resolving deltas: 100% (8980/8980), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 kB 862.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.1/123.1 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.8/59.8 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [17]:
!./ch train jsc-tiny jsc --max-epochs 1 --batch-size 256

[2023-11-15 14:28:54,099] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)
2023-11-15 14:28:55.027820: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-15 14:28:55.027896: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-15 14:28:55.027943: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-15 14:28:56.777347: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Seed set to 0
+-------------------------+----------------------+-----------------+----------------------+
| Name      

In [24]:
# Navigate to output directory and zip the output
!cd /content/mase-tools/mase_output/ && zip -r /content/jsc-tiny_classification_jsc_2023-11-15.zip ./jsc-tiny_classification_jsc_2023-11-15/

  adding: jsc-tiny_classification_jsc_2023-11-15/ (stored 0%)
  adding: jsc-tiny_classification_jsc_2023-11-15/hardware/ (stored 0%)
  adding: jsc-tiny_classification_jsc_2023-11-15/software/ (stored 0%)
  adding: jsc-tiny_classification_jsc_2023-11-15/software/training_ckpts/ (stored 0%)
  adding: jsc-tiny_classification_jsc_2023-11-15/software/training_ckpts/best.ckpt (deflated 55%)
  adding: jsc-tiny_classification_jsc_2023-11-15/software/training_ckpts/last.ckpt (deflated 55%)
  adding: jsc-tiny_classification_jsc_2023-11-15/software/tensorboard/ (stored 0%)
  adding: jsc-tiny_classification_jsc_2023-11-15/software/tensorboard/lightning_logs/ (stored 0%)
  adding: jsc-tiny_classification_jsc_2023-11-15/software/tensorboard/lightning_logs/version_0/ (stored 0%)
  adding: jsc-tiny_classification_jsc_2023-11-15/software/tensorboard/lightning_logs/version_0/events.out.tfevents.1700058539.1052d9865882.4462.0 (deflated 68%)
  adding: jsc-tiny_classification_jsc_2023-11-15/software/tensor

# General introduction

In this lab, you will learn how to use the software stack of MASE. There are in total 7 tasks you would need to finish, and 1 optional task.

# Turning you network to a graph

One specific feature of MASE is its capability to transform DL models to a computation graph using the [torch.fx](<https://pytorch.org/docs/stable/fx.html>) framework.


## Use the Transform functionality without CLI

This tutorial describes how to use the MASE transform functionality for a pre-trained model.

## Import related packages and machop

In [2]:
import sys
import logging
import os
from pathlib import Path
from pprint import pprint as pp

In [4]:
from chop.dataset import MaseDataModule, get_dataset_info
from chop.tools.logger import set_logging_verbosity, get_logger

from chop.passes.transforms.interface import save_node_meta_param_transform_pass
from chop.passes.analysis import (
    report_node_meta_param_analysis_pass,
    profile_statistics_analysis_pass,
)
from chop.passes import (
    add_common_metadata_analysis_pass,
    init_metadata_analysis_pass,
    add_software_metadata_analysis_pass,
)
from chop.tools.get_input import InputGenerator
from chop.passes.graph.mase_graph import MaseGraph

from chop.models import get_model_info, get_model

set_logging_verbosity("info")

[2023-11-15 14:16:47,561] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


INFO     Set logging level to info
INFO:chop:Set logging level to info


## Set up the dataset

Here we create a `MaseDataModule` using the `jsc` dataset from lab1. Note the `MaseDataModule` also requires the name of the model you plan to use data module with. In this case it is `jsc-tiny`.

In [5]:

logger = logging.getLogger("chop")
logger.setLevel(logging.INFO)

batch_size = 8
model_name = "jsc-tiny"
dataset_name = "jsc"


data_module = MaseDataModule(
    name=dataset_name,
    batch_size=batch_size,
    model_name=model_name,
    num_workers=0,
    # custom_dataset_cache_path="../../chop/dataset"
)
data_module.prepare_data()
data_module.setup()


## Set up the model

Here we load the `jsc-tiny` model from lab1. The follow code initialises an untrained model. We will later see how to load a pre-trained model from a checkpoint.

In [ ]:
model_info = get_model_info(model_name)
model = get_model(
    model_name,
    task="cls",
    dataset_info=data_module.dataset_info,
    pretrained=False,
    checkpoint=None)

Once the checkpoint files are in `mase_output`, you can load it:

In [16]:
model_info = get_model_info(model_name)
model = get_model(
    model_name,
    task="cls",
    dataset_info=data_module.dataset_info,
    pretrained=False,
    checkpoint="")


# Get a dummy data in
With the dataset module and model information, we can grab an input generator.

In [39]:
# get the input generator
input_generator = InputGenerator(
    data_module=data_module,
    model_info=model_info,
    task="cls",
    which_dataloader="train",
)

# a demonstration of how to feed an input value to the model
dummy_in = {"x": next(iter(data_module.train_dataloader()))[0]}
_ = model(**dummy_in)


## Generate a MaseGraph
We have two forms of passes: transform passes and analysis passes, both of them would require the model to be transferred into a MaseGraph to allow manipulation.

In [33]:
# generate the mase graph and initialize node metadata
mg = MaseGraph(model=model)


## Running an Analysis pass
Analysis pass DOES NOT change the graph

The following analysis passes are essential to prepare the graph for other passes

In [34]:
mg = init_metadata_analysis_pass(mg, None)
mg = add_common_metadata_analysis_pass(mg, dummy_in)
mg = add_software_metadata_analysis_pass(mg, None)


We will first run a simple graph analysis to understand the structure of the model.

In [35]:
# report graph is an analysis pass that shows you the detailed information in the graph
from chop.passes import report_graph_analysis_pass
_ = report_graph_analysis_pass(mg)


graph():
    %x : [num_users=1] = placeholder[target=x]
    %seq_blocks_0 : [num_users=1] = call_module[target=seq_blocks.0](args = (%x,), kwargs = {})
    %seq_blocks_1 : [num_users=1] = call_module[target=seq_blocks.1](args = (%seq_blocks_0,), kwargs = {})
    %seq_blocks_2 : [num_users=1] = call_module[target=seq_blocks.2](args = (%seq_blocks_1,), kwargs = {})
    %seq_blocks_3 : [num_users=1] = call_module[target=seq_blocks.3](args = (%seq_blocks_2,), kwargs = {})
    return seq_blocks_3Network overview:
{'placeholder': 1, 'get_attr': 0, 'call_function': 0, 'call_method': 0, 'call_module': 4, 'output': 1}
Layer types:
[]


## Running another Analysis pass: Profile statistics

The pass `profile_statistics_analysis_pass` collects statistics of parameters and activations, and save them to node's metadata.

Here is a list of all the supported statistics. Refer to the `__init__` of statistic classes in `chop.passes.analysis.statistical_profiler.stat` to check the args each stat class takes.

This is a more complex analysis than the previous pass, and thus it would require you to pass in additional arguments for this pass.

### Example: the range of weights & input activations of nodes

Say we want to collect the tensor-wise min-max range of the 1st `torch.nn.Linear` nodes' weights & bias, and the channel-wise 97% quantile min-max of the 1st `torch.nn.Linear` nodes' input activations. We can do the following:

In [11]:

pass_args = {
    "by": "type",                                                            # collect statistics by node name
    "target_weight_nodes": ["linear"],                                       # collect weight statistics for linear layers
    "target_activation_nodes": ["relu"],                                     # collect activation statistics for relu layers
    "weight_statistics": {
        "variance_precise": {"device": "cpu", "dims": "all"},                # collect precise variance of the weight
    },
    "activation_statistics": {
        "range_quantile": {"device": "cpu", "dims": "all", "quantile": 0.97} # collect 97% quantile of the activation range
    },
    "input_generator": input_generator,                                      # the input generator for feeding data to the model
    "num_samples": 32,                                                       # feed 32 samples to the model
}

We can use the `report_node_meta_param_analysis_pass` to inspect the collected statistics.

In [12]:
mg = profile_statistics_analysis_pass(mg, pass_args)
mg = report_node_meta_param_analysis_pass(mg, {"which": ("software",)})

Profiling act statistics: 100%|██████████| 4/4 [00:00<00:00, 103.00it/s]
INFO     Inspecting graph [add_common_meta_param_analysis_pass]
INFO:chop.passes.analysis.report.report_node:Inspecting graph [add_common_meta_param_analysis_pass]
INFO     
+--------------+--------------+---------------------+--------------+-------------------------------------------------------------------------------------------+
| Node name    | Fx Node op   | Mase type           | Mase op      | Software Param                                                                            |
+==============+==============+=====================+==============+===========================================================================================+
| x            | placeholder  | placeholder         | placeholder  | {'results': {'data_out_0': {'stat': {}}}}                                                 |
+--------------+--------------+---------------------+--------------+-----------------------------------------

## Running a Transform pass: Quantisation

As its name suggests, the transform pass would modify the `MaseGraph`.
Similar to the previous analysis pass example, we would need to first declare the configuration for the pass.

In [14]:
pass_args = {
    "by": "type",
    "default": {"config": {"name": None}},
    "linear": {
            "config": {
                "name": "integer",
                # data
                "data_in_width": 8,
                "data_in_frac_width": 4,
                # weight
                "weight_width": 8,
                "weight_frac_width": 4,
                # bias
                "bias_width": 8,
                "bias_frac_width": 4,
            }
    },
}

We can then proceed to apply the transformation, in this case, we kept the original graph on purpose, so that we can print a `diff`.

In [15]:
from chop.passes.transforms import (
    quantize_transform_pass,
    summarize_quantization_analysis_pass,
)
from chop.passes.utils import deepcopy_mase_graph


ori_mg = deepcopy_mase_graph(mg)
mg = quantize_transform_pass(mg, pass_args)
summarize_quantization_analysis_pass(ori_mg, mg, save_dir="quantize_summary")

INFO     Quantized graph histogram:
INFO:chop.passes.transforms.quantize.summary:Quantized graph histogram:
INFO     
| Original type   | OP           |   Total |   Changed |   Unchanged |
|-----------------+--------------+---------+-----------+-------------|
| BatchNorm1d     | batch_norm1d |       1 |         0 |           1 |
| Linear          | linear       |       1 |         1 |           0 |
| ReLU            | relu         |       2 |         0 |           2 |
| output          | output       |       1 |         0 |           1 |
| x               | placeholder  |       1 |         0 |           1 |
INFO:chop.passes.transforms.quantize.summary:
| Original type   | OP           |   Total |   Changed |   Unchanged |
|-----------------+--------------+---------+-----------+-------------|
| BatchNorm1d     | batch_norm1d |       1 |         0 |           1 |
| Linear          | linear       |       1 |         1 |           0 |
| ReLU            | relu         |       2 |         0 



# Exercises:

We have now seen how to:
1. Set up a dataset
2. Set up a model
3. Generate a `MaseGraph` from the model
4. Run Analysis and Transform passes on the `MaseGraph`

Now consider the following problems:

1. Explain the functionality of `report_graph_analysis_pass` and its printed jargons such as `placeholder`, `get_attr` ... You might find the doc of [torch.fx](https://pytorch.org/docs/stable/fx.html) useful.

2. What are the functionalities of `profile_statistics_analysis_pass` and `report_node_meta_param_analysis_pass` respectively?

## MASE OPs and MASE Types

MASE is designed to be a very high-level intermediate representation (IR), this is very different from the classic [LLVM IR](https://llvm.org/docs/LangRef.html) that you might be familiar with.

The following MASE Types are available:
(Note from Aaron: do we have a page somewhere that have summarized this?)


## A deeper dive into the quantisation transform

3. Explain why only 1 OP is changed after the `quantize_transform_pass` .

4. Write some code to traverse both `mg` and `ori_mg`, check and comment on the nodes in these two graphs. You might find the source code for the implementation of `summarize_quantization_analysis_pass` useful.

5. Perform the same quantisation flow to the bigger JSC network that you have trained in lab1. You must be aware that now the `pass_args` for your custom network might be different if you have used more than the `Linear` layer in your network.

6. Write code to show and verify that the weights of these layers are indeed quantised. You might need to go through the source code of the implementation of the quantisation pass and also the implementation of the [Quantized Layers](../../machop/chop/passes/transforms/quantize/quantized_modules/linear.py) .

## The command line interface

The same flow can also be executed on the command line throw the `transform` action.

```bash
# make sure you have the same printout
pwd
# it should show
# your_dir/mase-tools/machop

# enter the following command
./ch transform --config configs/examples/jsc_toy_by_type.toml --task cls --cpu=0
```
7. Load your own pre-trained JSC network, and perform perform the quantisation using the command line interface.

## \[Optional] Write your own pass

Many examples of existing passes are in the [source code](../..//machop/chop/passes/__init__.py), the [test files](../../machop/test/passes) for these passes also contain useful information on helping you to understand how these passes are used.

Implement a pass to count the number of FLOPs (floating-point operations) and BitOPs (bit-wise operations).

# Appendix
## Loading a pretrained model

Some of the tasks require use of the previously trained model from lab1. We can load the model from its saved checkpoint files.

First, add the checkpoint files of the model you wish to load to the `mase_output` directory:
- Upload a zip folder into Colab sidebar (e.g. jsc-tiny_classification_jsc_2023-11-15.zip).
- Unzip it to the `mase_output` directory by running:

In [27]:
# Create the mase_output directory if it doesn't already exist
!mkdir -p /content/mase-tools/mase_output/

# Unzip a model checkpoint and copy to the mase_output directory
!unzip /content/jsc-tiny_classification_jsc_2023-11-15.zip -d /content/mase-tools/mase_output/

Archive:  /content/jsc-tiny_classification_jsc_2023-11-15.zip
replace /content/mase-tools/mase_output/jsc-tiny_classification_jsc_2023-11-15/software/training_ckpts/best.ckpt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace /content/mase-tools/mase_output/jsc-tiny_classification_jsc_2023-11-15/software/training_ckpts/last.ckpt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace /content/mase-tools/mase_output/jsc-tiny_classification_jsc_2023-11-15/software/tensorboard/lightning_logs/version_0/events.out.tfevents.1700058539.1052d9865882.4462.0? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace /content/mase-tools/mase_output/jsc-tiny_classification_jsc_2023-11-15/software/tensorboard/lightning_logs/version_0/hparams.yaml? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


Once the checkpoint files are in `mase_output`, you can load it:

In [38]:
load_name = "../mase_output/jsc-tiny_classification_jsc_2023-11-15/software/training_ckpts/best.ckpt"

model = get_model(
    name=model_name,
    task="cls",
    dataset_info=data_module.dataset_info,
    pretrained=True,
    checkpoint=load_name,
)